In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!nvidia-smi

Thu Apr 25 16:57:27 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:0D:00.0 Off |                  Off |
| N/A   51C    P0   138W / 250W |  12512MiB / 24451MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:0E:00.0 Off |                  Off |
| N/A   37C    P0    55W / 250W |  21170MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:

from __future__ import absolute_import, division, print_function
from copy import deepcopy
from skimage.io import imread
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TEST_OPTIONS
from visualize import display_img_pairs_w_flows
import cv2

In [7]:
gpu_devices = ['/device:GPU:2']  
controller = '/device:GPU:2'
#можно использовать другие веса
ckpt_path = './pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-57000'

# Строим сетку, в соответствии с весами, которые вы используете

In [9]:
nn_opts = deepcopy(_DEFAULT_PWCNET_TEST_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = ckpt_path
nn_opts['batch_size'] = 1
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# We're running the PWC-Net-large model in quarter-resolution mode
# That is, with a 6 level pyramid, and upsampling of level 2 by 4 in each dimension as the final flow prediction
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2
# разрешение китти, для других датасетов изменить
nn_opts['adapt_info'] = (1, 375, 1242, 2)

In [10]:
nn = ModelPWCNet(mode='test', options=nn_opts)
nn.print_config()

Building model...
... model built.
Loading model checkpoint ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-57000 for eval or testing...

INFO:tensorflow:Restoring parameters from ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-57000
... model loaded

Model Configuration:
  verbose                True
  ckpt_path              ./pwcnet-lg-6-2-cyclic-sintel_finetuned/pwcnet.ckpt-57000
  x_dtype                <dtype: 'float32'>
  x_shape                [2, None, None, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [None, None, 2]
  gpu_devices            ['/device:GPU:2']
  controller             /device:GPU:2
  batch_size             1
  use_tf_data            True
  use_mixed_precision    False
  pyr_lvls               6
  flow_pred_lvl          2
  search_range           4
  use_dense_cx           True
  use_res_cx             True
  adapt_info             (1, 375, 1242, 2)
  mode                   test
  trainable params       14079050


# добавляем колонки

In [11]:
def get_image_number(line):
    return line.split('/')[-1][:-len('.jpg')]


def read_rgb_flow_from_csv(path, stride=1, type_flow='gt', name=''):
    df = pd.read_csv(os.path.join(path, 'df.csv'.format(stride)), index_col=0)
    path_to_second_rgb = df[['path_to_next_rgb']].values
    path_to_first_rgb = df[['path_to_rgb']].values
    path_to_flow = df[['optical_flow_{}_skip{}_{}'.format(type_flow, stride, name)]].values
    return np.hstack([path_to_first_rgb, path_to_second_rgb, path_to_flow])


def update_csv_of(path, filename, stride, name=''):
    df = pd.read_csv(str(path) + str(filename), index_col=0)
    df.insert(13, 'optical_flow_pwc_skip{}_{}'.format(stride, name))
    #df = df.drop(columns='optical_flow_pwc_skip1_finetune_sintel')
    #df = df.drop(columns='optical_flow_pwc_skip1_torch_finetune_sintel')
    for  (id1, row1) in df.iterrows():
        rgb1 = row1.path_to_rgb
        rgb2 = row1.path_to_next_rgb
        df.at[id1,'optical_flow_pwc_skip{}_{}'.format(stride, name)]= 'optical_flow_pwc_skip{}_{}/{}_{}.npy'.format(stride, name, get_image_number(rgb1),get_image_number(rgb2))
    df.to_csv(str(path) + '/' + str(filename))
    
    
def delete_column(path, filename, stride, name=''):
    df = pd.read_csv(str(path) + str(filename), index_col=0)
    df = df.drop(columns=name)
    df.to_csv(str(path) + '/' + str(filename))
finetune_sintel

In [25]:
import os
import pandas as pd
import glob 
path = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/'
allfolders = glob.glob( path +'/*')
for i in allfolders:
    print(i)
    if os.path.isfile( i + '/df.csv'):
        update_csv_of(i,'/df.csv', '1', 'finetune_sintel')

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/05
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/08
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/21
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/11
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/19
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/14
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/17
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/01
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/09
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/04
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/07
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/13
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/16
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/00
/Vol1/dbstore/datasets/KITTI_odometry_20

# Скрипт

In [12]:
from math import ceil
import cv2
def create_of(path, path_to_save, name):
    img_pairs=[]
    stride=1
    path_list = read_rgb_flow_from_csv(path, stride, type_flow='pwc', name=name)
    for i in path_list:
        img_pairs=[]
        prev_img = None
        im1_fn = path + str(i[0])
        im2_fn = path + str(i[1])
        flow_fn = path + str(i[2])
        print('created', i)
        if not os.path.isfile(flow_fn):
            image1, image2 = imread(im1_fn), imread(im2_fn)
            divisor = 64.
            H = image1.shape[0]
            W = image1.shape[1]
            H_ = int(ceil(H / divisor) * divisor)
            W_ = int(ceil(W / divisor) * divisor)
            image1 = cv2.resize(image1, ( W_, H_))
            image2 = cv2.resize(image2, ( W_, H_))
            img_pairs.append((image1, image2))
            nn.predict_from_img_pairs_save(img_pairs, flow_fn,  batch_size=1, verbose=False)

# Запуск

In [ ]:
import os
import pandas as pd
import numpy as np
import glob 
path = '/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/'
allfolders = glob.glob( path +"/*/")
#list_temp = []
path_to_save = "/optical_flow_pwc_skip1_finetune_sintel"
for i in allfolders:
    #print(i)
    if os.path.isfile( i +  "/df.csv"):
        if not os.path.exists(i +  path_to_save):
            os.makedirs(i + path_to_save)
        create_of(i, path_to_save, 'finetune_sintel')

        
        #nn.predict_from_img_pairs(img_pairs, batch_size=1, verbose=False)

created ['image_2/000000.png' 'image_2/000001.png'
 'optical_flow_pwc_skip1_finetune_sintel/000000_000001.npy']
created ['image_2/000001.png' 'image_2/000002.png'
 'optical_flow_pwc_skip1_finetune_sintel/000001_000002.npy']
created ['image_2/000002.png' 'image_2/000003.png'
 'optical_flow_pwc_skip1_finetune_sintel/000002_000003.npy']
created ['image_2/000003.png' 'image_2/000004.png'
 'optical_flow_pwc_skip1_finetune_sintel/000003_000004.npy']
created ['image_2/000004.png' 'image_2/000005.png'
 'optical_flow_pwc_skip1_finetune_sintel/000004_000005.npy']
created ['image_2/000005.png' 'image_2/000006.png'
 'optical_flow_pwc_skip1_finetune_sintel/000005_000006.npy']
created ['image_2/000006.png' 'image_2/000007.png'
 'optical_flow_pwc_skip1_finetune_sintel/000006_000007.npy']
created ['image_2/000007.png' 'image_2/000008.png'
 'optical_flow_pwc_skip1_finetune_sintel/000007_000008.npy']
created ['image_2/000008.png' 'image_2/000009.png'
 'optical_flow_pwc_skip1_finetune_sintel/000008_00000

created ['image_2/000123.png' 'image_2/000124.png'
 'optical_flow_pwc_skip1_finetune_sintel/000123_000124.npy']
created ['image_2/000124.png' 'image_2/000125.png'
 'optical_flow_pwc_skip1_finetune_sintel/000124_000125.npy']
created ['image_2/000125.png' 'image_2/000126.png'
 'optical_flow_pwc_skip1_finetune_sintel/000125_000126.npy']
created ['image_2/000126.png' 'image_2/000127.png'
 'optical_flow_pwc_skip1_finetune_sintel/000126_000127.npy']
created ['image_2/000127.png' 'image_2/000128.png'
 'optical_flow_pwc_skip1_finetune_sintel/000127_000128.npy']
created ['image_2/000128.png' 'image_2/000129.png'
 'optical_flow_pwc_skip1_finetune_sintel/000128_000129.npy']
created ['image_2/000129.png' 'image_2/000130.png'
 'optical_flow_pwc_skip1_finetune_sintel/000129_000130.npy']
created ['image_2/000130.png' 'image_2/000131.png'
 'optical_flow_pwc_skip1_finetune_sintel/000130_000131.npy']
created ['image_2/000131.png' 'image_2/000132.png'
 'optical_flow_pwc_skip1_finetune_sintel/000131_00013

created ['image_2/000269.png' 'image_2/000270.png'
 'optical_flow_pwc_skip1_finetune_sintel/000269_000270.npy']
created ['image_2/000270.png' 'image_2/000271.png'
 'optical_flow_pwc_skip1_finetune_sintel/000270_000271.npy']
created ['image_2/000271.png' 'image_2/000272.png'
 'optical_flow_pwc_skip1_finetune_sintel/000271_000272.npy']
created ['image_2/000272.png' 'image_2/000273.png'
 'optical_flow_pwc_skip1_finetune_sintel/000272_000273.npy']
created ['image_2/000273.png' 'image_2/000274.png'
 'optical_flow_pwc_skip1_finetune_sintel/000273_000274.npy']
created ['image_2/000274.png' 'image_2/000275.png'
 'optical_flow_pwc_skip1_finetune_sintel/000274_000275.npy']
created ['image_2/000275.png' 'image_2/000276.png'
 'optical_flow_pwc_skip1_finetune_sintel/000275_000276.npy']
created ['image_2/000276.png' 'image_2/000277.png'
 'optical_flow_pwc_skip1_finetune_sintel/000276_000277.npy']
created ['image_2/000277.png' 'image_2/000278.png'
 'optical_flow_pwc_skip1_finetune_sintel/000277_00027

created ['image_2/000416.png' 'image_2/000417.png'
 'optical_flow_pwc_skip1_finetune_sintel/000416_000417.npy']
created ['image_2/000417.png' 'image_2/000418.png'
 'optical_flow_pwc_skip1_finetune_sintel/000417_000418.npy']
created ['image_2/000418.png' 'image_2/000419.png'
 'optical_flow_pwc_skip1_finetune_sintel/000418_000419.npy']
created ['image_2/000419.png' 'image_2/000420.png'
 'optical_flow_pwc_skip1_finetune_sintel/000419_000420.npy']
created ['image_2/000420.png' 'image_2/000421.png'
 'optical_flow_pwc_skip1_finetune_sintel/000420_000421.npy']
created ['image_2/000421.png' 'image_2/000422.png'
 'optical_flow_pwc_skip1_finetune_sintel/000421_000422.npy']
created ['image_2/000422.png' 'image_2/000423.png'
 'optical_flow_pwc_skip1_finetune_sintel/000422_000423.npy']
created ['image_2/000423.png' 'image_2/000424.png'
 'optical_flow_pwc_skip1_finetune_sintel/000423_000424.npy']
created ['image_2/000424.png' 'image_2/000425.png'
 'optical_flow_pwc_skip1_finetune_sintel/000424_00042

created ['image_2/000560.png' 'image_2/000561.png'
 'optical_flow_pwc_skip1_finetune_sintel/000560_000561.npy']
created ['image_2/000561.png' 'image_2/000562.png'
 'optical_flow_pwc_skip1_finetune_sintel/000561_000562.npy']
created ['image_2/000562.png' 'image_2/000563.png'
 'optical_flow_pwc_skip1_finetune_sintel/000562_000563.npy']
created ['image_2/000563.png' 'image_2/000564.png'
 'optical_flow_pwc_skip1_finetune_sintel/000563_000564.npy']
created ['image_2/000564.png' 'image_2/000565.png'
 'optical_flow_pwc_skip1_finetune_sintel/000564_000565.npy']
created ['image_2/000565.png' 'image_2/000566.png'
 'optical_flow_pwc_skip1_finetune_sintel/000565_000566.npy']
created ['image_2/000566.png' 'image_2/000567.png'
 'optical_flow_pwc_skip1_finetune_sintel/000566_000567.npy']
created ['image_2/000567.png' 'image_2/000568.png'
 'optical_flow_pwc_skip1_finetune_sintel/000567_000568.npy']
created ['image_2/000568.png' 'image_2/000569.png'
 'optical_flow_pwc_skip1_finetune_sintel/000568_00056

...

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002292_002293.npy
created ['image_2/002293.png' 'image_2/002294.png'
 'optical_flow_pwc_skip1_finetune_sintel/002293_002294.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002293_002294.npy
created ['image_2/002294.png' 'image_2/002295.png'
 'optical_flow_pwc_skip1_finetune_sintel/002294_002295.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002294_002295.npy
created ['image_2/002295.png' 'image_2/002296.png'
 'optical_flow_pwc_skip1_finetune_sintel/002295_002296.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002295_002296.npy
created ['image_2/002296.png' 'image_2/002297.png'
 'optical_flow_pwc_skip1_finetune_sintel/002296_002297.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/opti

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002328_002329.npy
created ['image_2/002329.png' 'image_2/002330.png'
 'optical_flow_pwc_skip1_finetune_sintel/002329_002330.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002329_002330.npy
created ['image_2/002330.png' 'image_2/002331.png'
 'optical_flow_pwc_skip1_finetune_sintel/002330_002331.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002330_002331.npy
created ['image_2/002331.png' 'image_2/002332.png'
 'optical_flow_pwc_skip1_finetune_sintel/002331_002332.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002331_002332.npy
created ['image_2/002332.png' 'image_2/002333.png'
 'optical_flow_pwc_skip1_finetune_sintel/002332_002333.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/opti

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002364_002365.npy
created ['image_2/002365.png' 'image_2/002366.png'
 'optical_flow_pwc_skip1_finetune_sintel/002365_002366.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002365_002366.npy
created ['image_2/002366.png' 'image_2/002367.png'
 'optical_flow_pwc_skip1_finetune_sintel/002366_002367.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002366_002367.npy
created ['image_2/002367.png' 'image_2/002368.png'
 'optical_flow_pwc_skip1_finetune_sintel/002367_002368.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002367_002368.npy
created ['image_2/002368.png' 'image_2/002369.png'
 'optical_flow_pwc_skip1_finetune_sintel/002368_002369.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/opti

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002400_002401.npy
created ['image_2/002401.png' 'image_2/002402.png'
 'optical_flow_pwc_skip1_finetune_sintel/002401_002402.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002401_002402.npy
created ['image_2/002402.png' 'image_2/002403.png'
 'optical_flow_pwc_skip1_finetune_sintel/002402_002403.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002402_002403.npy
created ['image_2/002403.png' 'image_2/002404.png'
 'optical_flow_pwc_skip1_finetune_sintel/002403_002404.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002403_002404.npy
created ['image_2/002404.png' 'image_2/002405.png'
 'optical_flow_pwc_skip1_finetune_sintel/002404_002405.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/opti

/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002436_002437.npy
created ['image_2/002437.png' 'image_2/002438.png'
 'optical_flow_pwc_skip1_finetune_sintel/002437_002438.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002437_002438.npy
created ['image_2/002438.png' 'image_2/002439.png'
 'optical_flow_pwc_skip1_finetune_sintel/002438_002439.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002438_002439.npy
created ['image_2/002439.png' 'image_2/002440.png'
 'optical_flow_pwc_skip1_finetune_sintel/002439_002440.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/optical_flow_pwc_skip1_finetune_sintel/002439_002440.npy
created ['image_2/002440.png' 'image_2/002441.png'
 'optical_flow_pwc_skip1_finetune_sintel/002440_002441.npy']
/Vol1/dbstore/datasets/KITTI_odometry_2012/dataset/sequences/02/opti